<a href="https://colab.research.google.com/github/Matrix7043/Machine_learning101/blob/main/Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import typing
import pandas as pd
import numpy as np
from numpy import ndarray

In [ ]:
def sigmoid(X: ndarray) -> ndarray:
    return 1/(1 + np.exp(-1.0 * X))


In [ ]:
df = pd.read_csv('/content/sample_data/diamond.csv')
df1 = df[['carat', 'x', 'y', 'z', 'depth']].head(100)
df2 = df[['price']].head(100)
y = np.array(df2['price'])
x = np.transpose(np.array([df1['carat'],df1['depth'], df1['x'], df1['y'], df1['z']]))
y = y.reshape(100, 1)

In [ ]:
def forward_propagation(X_batch: ndarray,
                        Y_batch: ndarray,
                        weights: dict[str, ndarray]) -> tuple[dict[str, ndarray], float]:
    #W1 = np.transpose(weights['W1'])
    #W2 = np.transpose(weights['W2'])

    M1 = np.dot(X_batch, weights['W1'])
    # print("M1", M1.shape)

    N1 = M1 + weights['B1']
    # print("N1", N1.shape)

    O1 = sigmoid(N1)
    # print("O1", O1.shape)

    M2 = np.dot(O1, weights['W2'])
    # print("M2", M2.shape)

    P = M2 + weights['B2']
    # print("P", P.shape)

    L = np.mean(np.power((Y_batch - P), 2))

    forward_info: dict[str, ndarray] = {}
    forward_info['X'] = X_batch
    forward_info['Y'] = Y_batch
    forward_info['W1'] = weights['W1']
    forward_info['W2'] = weights['W2']
    forward_info['M1'] = M1
    forward_info['M2'] = M2
    forward_info['N1'] = N1
    forward_info['O1'] = O1
    forward_info['P'] = P

    return forward_info, L


In [ ]:
def back_propogation(forward_info: dict[str, ndarray],
                     weights: dict[str, ndarray]) -> dict[str, ndarray]:

    dLdP = -1 * (forward_info['Y'] - forward_info['P'])
    # print("Y", forward_info['Y'])
    # print("P", forward_info['P'])
    # print("dLdP", dLdP.shape)

    dPdM2 = np.ones_like(forward_info['M2'])
    # print("dPdM2", dPdM2.shape)

    dM2dO1 = np.transpose(forward_info['W2'])
    # print("dM2dO1", dM2dO1.shape)

    dO1dN1 = sigmoid(forward_info['N1'] * (1 - sigmoid(forward_info['N1'])))
    # print("dO1dN1", dO1dN1.shape)

    dN1dM1 = np.ones_like(forward_info['M1'])
    # print("dN1dM1", dN1dM1.shape)

    dM1dW1 = np.transpose(forward_info['X'])
    # print("dM1dW1", dM1dW1.shape)

    dN1dB1 = np.ones_like(weights['B1'])
    # print("dN1dB1", dN1dB1.shape)

    dM2dW2 = np.transpose(forward_info['O1'])
    # print("dM2dW1", dM2dW2.shape)

    dPdB2 = np.ones_like(weights['B2'])
    # print("dPdB2", dPdB2.shape)

    dLdM2 = dLdP * dPdM2
    # print("dLdM2", dLdM2.shape)

    dLdO1 = np.dot(dLdM2, dM2dO1)
    # print("dLdO1", dLdO1.shape)

    dLdM1 = (dLdO1 * dO1dN1) * dN1dM1
    # print("dLdM1", dLdM1.shape)

    # Important

    dLdW1 = np.dot(dM1dW1, dLdM1)

    dLdW2 = np.dot(dM2dW2, dLdP)

    dLdB1 = ((dLdO1 * dO1dN1) * dN1dB1).sum(axis=0)

    dLdB2 = (dLdP * dPdB2).sum(axis=0)

    loss_gradient: dict[str, ndarray] = {}
    loss_gradient['W1'] = dLdW1
    loss_gradient['W2'] = dLdW2
    loss_gradient['B1'] = dLdB1
    loss_gradient['B2'] = dLdB2

    return loss_gradient


In [ ]:
def train(X_train: ndarray,
          Y_train: ndarray,
          learning_rate: float = 0.000001,
          hidden_size: int = 13,
          iterations: int = 50000) -> dict[str, ndarray]:

    #print("Y", Y_train.shape)
    #print("X", X_train.shape)

    def init_weights(input_size: int,
                     hidden_size: int) -> dict[str, ndarray]:

              weights: dict[str, ndarray] = {}
              weights['W1'] = np.random.randn(input_size, hidden_size)
              weights['B1'] = np.random.randn(1, hidden_size)
              weights['W2'] = np.random.randn(hidden_size, 1)
              weights['B2'] = np.random.randn(1)
              #print("W1", weights['W1'].shape)
              #print("B1", weights['B1'].shape)
              #print("W2", weights['W2'].shape)
              #print("B2", weights['B2'].shape)
              return weights

    weights = init_weights(X_train.shape[1], hidden_size)

    for i in range(iterations):

        forward_info, loss = forward_propagation(X_train, Y_train, weights)
        loss_grads = back_propogation(forward_info, weights)

        if i%10 == 0:
            print(loss)

        for key in weights.keys():
            weights[key] -= learning_rate * loss_grads[key]

    return weights

In [ ]:
theta = train(x, y)

942137.3035817039
934388.8737914233
931750.6954677264
929128.412933158
926521.851504813
923930.8986926237
921355.4545157694
918795.4232338938
916250.7111107759
913721.2256284266
911206.8751592743
908707.5688112582
906223.2163470743
903753.7281384495
901299.01513827
898858.9888624008
896433.5613770309
894022.6452893247
891626.1537401128
889244.0003978986
886876.0994537263
884522.3656166381
882182.7141095396
879857.0606653577
877545.3215234098
875247.4134259332
872963.253614737
870692.759827946
868435.8502968238
866192.4437426553
863962.4593736818
861745.8168820803
859542.4364409802
857352.238701514
855175.1447898985
853011.0763045433
850859.9553131841
848721.7043500413
846596.2464129992
844483.5049608082
842383.4039103057
840295.8676336587
838220.8209556229
836158.1891508234
834107.8979410502
832069.8734925736
830044.0424134767
828030.3317510042
826028.6689889282
824038.9820449312
822061.1992680046
820095.2494358635
818141.0617523789
816198.5658450231
814267.6917623326
812348.3699713874